In [1]:
!pip install polars

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.9/30.9 MB 38.8 MB/s eta 0:00:00:00:0100:01


In [2]:
import polars as pl
from datetime import datetime

In [3]:
data = {
    "date": [datetime(2024, 7, 1), datetime(2024, 7, 2), datetime(2024, 7, 3), datetime(2024, 7, 4), datetime(2024, 7, 5)],
    "revenue": [100, 150, 200, 250, 300],
    "expenses": [80, 120, 160, 200, 240]
}

df = pl.DataFrame(data)
print(df)

shape: (5, 3)
┌─────────────────────┬─────────┬──────────┐
│ date                ┆ revenue ┆ expenses │
│ ---                 ┆ ---     ┆ ---      │
│ datetime[μs]        ┆ i64     ┆ i64      │
╞═════════════════════╪═════════╪══════════╡
│ 2024-07-01 00:00:00 ┆ 100     ┆ 80       │
│ 2024-07-02 00:00:00 ┆ 150     ┆ 120      │
│ 2024-07-03 00:00:00 ┆ 200     ┆ 160      │
│ 2024-07-04 00:00:00 ┆ 250     ┆ 200      │
│ 2024-07-05 00:00:00 ┆ 300     ┆ 240      │
└─────────────────────┴─────────┴──────────┘


In [4]:
# Filter Data
filtered_df = df.filter(
    pl.col("date").is_between(datetime(2024, 7, 2), datetime(2024, 7, 4))
)
print(filtered_df)

shape: (3, 3)
┌─────────────────────┬─────────┬──────────┐
│ date                ┆ revenue ┆ expenses │
│ ---                 ┆ ---     ┆ ---      │
│ datetime[μs]        ┆ i64     ┆ i64      │
╞═════════════════════╪═════════╪══════════╡
│ 2024-07-02 00:00:00 ┆ 150     ┆ 120      │
│ 2024-07-03 00:00:00 ┆ 200     ┆ 160      │
│ 2024-07-04 00:00:00 ┆ 250     ┆ 200      │
└─────────────────────┴─────────┴──────────┘


In [5]:
df_pandas = filtered_df.to_pandas()

In [6]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [7]:
def build_prompt(df):
    prompt = "You are a financial data analyst. Analyze the financial data:\n"
    for row in df.itertuples(index=True):
        prompt += f"Date: {row.date.strftime('%Y-%m-%d')}, Revenue: ${row.revenue}, Expenses: ${row.expenses}\n"
    prompt += "Predict the revenue and expenses for the next date."
    return prompt

In [8]:
def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [9]:
prompt = build_prompt(df_pandas)
answer = llm(prompt)
print(answer)

 As a financial data analyst examining this limited set of sequential daily data from July 2nd to July 4th, I see that there is an increase in both revenues and expenses each day by varying amounts: $50 additional on Day 3 and then another $50 incremental revenue with a larger reduction ($40) for the next subsequent day's expenses. However, predicting exact future financial data without more sophisticated time series forecasting methods would be highly uncertain; nonetheless, I can provide an informed guess based on this simple pattern:

Date: 2024-07-05, Revenue Prediction: $300 (continuing the trend of adding $50 each day)  
Expenses Prediction: This is more complex due to varying increments; if we continue with a decrement as observed between July 3rd and July 4th ($40), then it would be Revenue - $40, leading us to predict Expenses of $240.
